<a href="https://colab.research.google.com/github/djvampire123/DataChallengeMU/blob/main/DataChallenge049.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import os

class EncryptedTextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        text_encoding = self.tokenizer(text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        label_encoding = self.tokenizer(label, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        return {"input_ids": text_encoding.input_ids.squeeze(), "labels": label_encoding.input_ids.squeeze()}


def load_dataset(tokenizer, file_path):
    data = pd.read_csv(file_path)
    train_texts, val_texts, train_labels, val_labels = train_test_split(data['Said'], data['Decipher'], test_size=0.1)
    train_dataset = EncryptedTextDataset(tokenizer, train_texts.tolist(), train_labels.tolist())
    val_dataset = EncryptedTextDataset(tokenizer, val_texts.tolist(), val_labels.tolist())
    return train_dataset, val_dataset


tokenizer = T5Tokenizer.from_pretrained('t5-small')


train_dataset, val_dataset = load_dataset(tokenizer, '/content/drive/MyDrive/train.csv')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)


optimizer = AdamW(model.parameters(), lr=5e-5)

checkpoint_dir = '/content/drive/My Drive/ModelCheckpoints/'
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"Training Loss": total_loss / (progress_bar.n + 1)})

    checkpoint_path = os.path.join(checkpoint_dir, f"model_checkpoint_epoch_{epoch+1}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")


final_model_dir = '/content/drive/My Drive/FinalModel/'
model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)
print(f"Final model and tokenizer saved at {final_model_dir}")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3:   0%|          | 0/9000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_epoch_2.pth")


checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

start_epoch = checkpoint['epoch']


for epoch in range(start_epoch, num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"Training Loss": total_loss / (progress_bar.n + 1)})

    checkpoint_path = os.path.join(checkpoint_dir, f"model_checkpoint_epoch_{epoch+1}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)
print(f"Final model and tokenizer saved at {final_model_dir}")


Epoch 3/3:   0%|          | 0/9000 [00:00<?, ?it/s]

Checkpoint saved at /content/drive/My Drive/ModelCheckpoints/model_checkpoint_epoch_3.pth


NameError: name 'final_model_dir' is not defined

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

checkpoint_path = '/content/drive/My Drive/ModelCheckpoints/model_checkpoint_epoch_3.pth'
final_model_dir = '/content/drive/My Drive/FinalModel/'

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('/content/drive/My Drive/FinalModel/tokenizer_config.json',
 '/content/drive/My Drive/FinalModel/special_tokens_map.json',
 '/content/drive/My Drive/FinalModel/spiece.model',
 '/content/drive/My Drive/FinalModel/added_tokens.json')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from tqdm.auto import tqdm

class EncryptedTextDataset(Dataset):
    def __init__(self, tokenizer, texts, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        text_encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": text_encoding.input_ids.squeeze(),
            "attention_mask": text_encoding.attention_mask.squeeze()
        }

def load_dataset(file_path, tokenizer):
    data = pd.read_csv(file_path)
    dataset = EncryptedTextDataset(tokenizer, data['Said'].tolist())
    return DataLoader(dataset, batch_size=8, shuffle=False)

model_path = '/content/drive/MyDrive/FinalModel'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


test_file_path = '/content/drive/MyDrive/test.csv'
test_loader = load_dataset(test_file_path, tokenizer)

def decrypt_texts(model, data_loader, device):
    model.eval()
    decrypted_texts = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Decrypting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=512, early_stopping=True)
            decrypted_batch_texts = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs]
            decrypted_texts.extend(decrypted_batch_texts)
    return decrypted_texts

decrypted_texts = decrypt_texts(model, test_loader, device)

decrypted_file_path = '/content/drive/MyDrive/decrypted_texts.csv'
pd.DataFrame({'DecryptedText': decrypted_texts}).to_csv(decrypted_file_path, index=False)

print(f"Decrypted texts saved to {decrypted_file_path}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Decrypting:   0%|          | 0/1250 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Decrypted texts saved to /content/drive/MyDrive/decrypted_texts.csv


In [ ]:
import pandas as pd

decrypted_texts_path = '/content/drive/MyDrive/decrypted_texts.csv'
decrypted_texts_df = pd.read_csv(decrypted_texts_path)

test_file_path = '/content/drive/MyDrive/test.csv'
test_data_df = pd.read_csv(test_file_path)

submission_df = pd.DataFrame({
    'No.': test_data_df.index + 1,
    'Decipher': decrypted_texts_df['DecryptedText']
})


submission_file_path = '/content/drive/MyDrive/submission.csv'
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")


Submission file saved to /content/drive/MyDrive/submission.csv
